# Step 1 - Extract and translate sentences from video

In [ ]:
import os
import sys
import pandas as pd
import assemblyai as aai
import os
from pytube import YouTube

aai.settings.api_key = '4ba97f247dd44f86b2c51a29f14caa26'

In [ ]:
aai.settings.api_key = ASSEMBLYAI_API_KEY

# Path to your audio file
audio_file = "../Data/La isla de las tentaciones Temporada 7 capitulo 7.mp3"

# Configure transcription with Spanish language
config = aai.TranscriptionConfig(language_code="es")
transcriber = aai.Transcriber(config=config)

# Transcribe the audio
transcript = transcriber.transcribe(audio_file)

# Check for errors
if transcript.status == aai.TranscriptStatus.error:
    print(f"Transcription failed: {transcript.error}")
    exit(1)

# Create lists to store data
sentences = []
start_times = []
end_times = []

# Extract sentences with their timestamps
sentence_objects = transcript.get_sentences()
for sentence in sentence_objects:
    sentences.append(sentence.text)
    start_times.append(sentence.start)  # Start time in milliseconds
    end_times.append(sentence.end)      # End time in milliseconds

# Convert to DataFrame for easier manipulation
transcript_df = pd.DataFrame({
    'sentence': sentences,
    'start_time_ms': start_times,
    'end_time_ms': end_times
})

# Convert milliseconds to a more readable format
transcript_df['start_time'] = transcript_df['start_time_ms'].apply(
    lambda ms: f"{int(ms/60000):02d}:{int((ms%60000)/1000):02d}.{int(ms%1000):03d}"
)
transcript_df['end_time'] = transcript_df['end_time_ms'].apply(
    lambda ms: f"{int(ms/60000):02d}:{int((ms%60000)/1000):02d}.{int(ms%1000):03d}"
)

# Format the DataFrame
transcript_df = transcript_df[['start_time', 'end_time', 'sentence']]

# Rename the columns as per template
transcript_df.columns = ['Start Time', 'End Time', 'Sentence']

# Display the first few rows to check
print(transcript_df.head())

# Save to CSV for the next step in your pipeline
output_path = "../task_11"
#transcript_df.to_csv(output_path, index=False)

#print(f"Extracted {len(sentences)} sentences with timestamps and saved to {output_path}")

# Step 2: Translate the sentences to English

In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

# === Load model and tokenizer ===
model_path = "/Users/Buas/Desktop/Data Start/GitHub/Ano 2/2024-25c-fai2-adsai-DeuzaVarela235065/task_11/translation_model"  # <-- Replace with your local model path
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_path)


/Users/Buas/Desktop/Data Start/GitHub/Ano 2/2024-25c-fai2-adsai-DeuzaVarela235065/keras_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/Buas/Desktop/Data Start/GitHub/Ano 2/2024-25c-fai2-adsai-DeuzaVarela235065/keras_env/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at /Users/Buas/Desktop/Data Start/GitHub/Ano 2/2024-25c-fai2-adsai-DeuzaVarela235065/task_11/translation_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictio

In [ ]:
# === Load Excel file ===
input_path = "./extracted_sentences.csv"   
df = pd.read_csv(input_path)

In [6]:
# === Load Excel file ===
input_path = "./extracted_sentences.csv"   
df = pd.read_csv(input_path)

# === Translate a column of sentences ===
def translate(text):

    # Tokenize input text
    inputs = tokenizer.encode(text, return_tensors="tf", padding=True, truncation=True, max_length=256)
    # Generate translation
    outputs = model.generate(inputs, max_length=256)
    # Decode the output
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Apply translation to the column (replace 'text' with your actual column name)
df['translated_sentence'] = df['Sentence'].apply(translate)

# === Save to new Excel file ===
output_path = "./extracted_sentences.csv"   # <-- Replace with desired output path
df.to_csv(output_path, index=False)